In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
# os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")


In [2]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

/home/maniram/workspace/python/langchain-projects/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
agent = create_agent(
    model="gpt-4o-mini",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("messages",10),
            keep=("messages",4)
        )
    ]
)

In [4]:
## Run with thread id

config = {"configurable":{"thread_id":"test-1"}}

In [5]:
questions=[
    "what is 2+2 ?",
    "what is 10*5 ?",
    "what is 14/3 ?",
    "what is 12-89 ?",
    "what is 3*3 ?",
]

for q in questions:
    response=agent.invoke({"messages":[HumanMessage(content=q)]},config)
    print(f"Messages :  {response}")
    print(f"Messages : {len(response['messages'])}")

Messages :  {'messages': [HumanMessage(content='what is 2+2 ?', additional_kwargs={}, response_metadata={}, id='2dd86e5a-ec65-47eb-b9b0-6b6bb6a6abcb'), AIMessage(content='2 + 2 equals 4.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CvhqzjIjlFGSwK9pYoBnCjfgpAguT', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9d38-d22b-79b3-9286-b2dbc8a143f2-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details

In [ ]:
import os
llamalib = os.getenv('OLLAMA_LLM_LIBRARY', 'localhost')
llamalib
# llamalib = os.environ['OLLAMA_LLM_LIBRARY']
llamalib

KeyError: 'OLLAMA_LLM_LIBRARY'

### TokenSize

In [15]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

def search_hotels(city:str) -> str :
    """Search hotels - return long response to use more tokens"""
    return f"""Hotels in {city} :
        1. Grand Hotel - 5 start,$390/night, spa, pool,gym
        2. City Inn - 4 star, $200/night , business center
        3. Budget Stay - 33 star, $75/night free wifi"""

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("tokens",550),
            keep=("tokens",200)
        )
    ]
)
config={"configurable":{"thread_id":"test-1"}}

#Token Counter
def count_tokens(messages):
    total_chars=sum(len(str(m.content)) for m in messages)
    return total_chars//4 # 4 chars = 1 token

In [16]:
cities=["Paris","London","Tokyo","New York","Dubai","Singapore"]

for city in cities:
    response=agent.invoke(
        {"messages":[HumanMessage(content=f"Find hotels in {city}")]},
        config=config
    )
    tokens=count_tokens(response["messages"])
    print(f"{city}: ~{tokens} tokens, {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

Paris: ~146 tokens, 4 messages
[HumanMessage(content='Find hotels in Paris', additional_kwargs={}, response_metadata={}, id='1fbe68c8-6ee9-4507-8ec6-8ae78c234a1e'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 53, 'total_tokens': 68, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cvi7EYCKQxgWjV76q5U0JWhlodvqc', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d48-2eaf-7733-87b9-3d7cfd70e167-0', tool_calls=[{'name': 'search_hotels', 'args': {'city': 'Paris'}, 'id': 'call_L904CButzW4r6oGxoBUzx5ka', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 53

### Fraction

In [18]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langchain_core.tools import tool
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

def search_hotels(city:str) -> str :
    """Search hotels - return long response to use more tokens"""
    return f"""Hotels in {city} :
        1. Grand Hotel - 5 start,$390/night, spa, pool,gym
        2. City Inn - 4 star, $200/night , business center
        3. Budget Stay - 33 star, $75/night free wifi"""

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_hotels],
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",
            trigger=("fraction",.005),
            keep=("fraction",.002)
        )
    ]
)
config={"configurable":{"thread_id":"test-1"}}

#Token Counter
def count_tokens(messages):
    total_chars=sum(len(str(m.content)) for m in messages)
    return total_chars//4 # 4 chars = 1 token

In [20]:
cities=["Paris","London","Tokyo","New York","Dubai","Singapore"]

for city in cities:
    response=agent.invoke(
        {"messages":[HumanMessage(content=f"Find hotels in {city}")]},
        config=config
    )
    tokens=count_tokens(response["messages"])
    fraction = tokens/12800  # gpt-40-mini context
    print(f"{city}: ~{tokens} tokens ({fraction:.4%}), {len(response['messages'])} messages")
    print(f"{(response['messages'])}")

Paris: ~491 tokens (3.8359%), 7 messages
[HumanMessage(content='Here is a summary of the conversation to date:\n\nUser requested hotel information for various cities: \n\n**Hotel Information:**\n- **Paris:** \n  - Grand Hotel - 5 stars, $390/night, features a spa, pool, and gym.\n  - City Inn - 4 stars, $200/night, includes a business center.\n  - Budget Stay - 3 stars, $75/night, offers free WiFi.\n  \n- **London:** \n  - Grand Hotel - 5 stars, $390/night, features a spa, pool, and gym.\n  - City Inn - 4 stars, $200/night, includes a business center.\n  - Budget Stay - 3 stars, $75/night, offers free WiFi.\n\n- **Tokyo:** \n  - Grand Hotel - 5 stars, $390/night, features a spa, pool, and gym.\n  - City Inn - 4 stars, $200/night, includes a business center.\n  - Budget Stay - 3 stars, $75/night, offers free WiFi.\n\n- **New York:** \n  - Grand Hotel - 5 stars, $390/night, features a spa, pool, and gym.\n  - City Inn - 4 stars, $200/night, includes a business center.\n  - Budget Stay - 

### Human in the loop middleware

In [29]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

def read_email_tool(email_id:str):
    """Mock function to read an email by its id"""
    return f"Email content for {email_id}"

def send_email_tool(receipient:str,subject:str,body:str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {receipient} with subject `{subject}`"

In [30]:

agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approve","edit","reject"]
                },
                "read_email_tool":False,
            }
        )
        
    ]
)


In [31]:
config={"configurable":{"thread_id":"test-approve"}}

#Step 1 - request
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'")]},
    config=config
)


In [32]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='9722587a-de5a-4884-a77f-74f33a6d22e0'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CviwIuJIaLxaCiATUDwzI4H0N8d7j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d78-7c56-7732-a95d-d53e04f0aefe-0', tool_calls=[{'name': 'send_email_tool', 'args': {'receipient': 'john@test.com', 'subject': 'How are you?', 'body': 'How are you?'}, 'id': 'call_GtiIYeeemYI

In [33]:
from langgraph.types import Command
# Step 2 Aprove
if "__interrupt__" in result:
    print("Paused! Approving..")
    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"approve"}
                ]
        }),
        config=config
    )

    print(f"Result : {result['messages'][-1].content}")


Paused! Approving..
Result : Email has been sent to john@test.com with the subject "How are you?" and the body "How are you?".


In [34]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='9722587a-de5a-4884-a77f-74f33a6d22e0'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CviwIuJIaLxaCiATUDwzI4H0N8d7j', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d78-7c56-7732-a95d-d53e04f0aefe-0', tool_calls=[{'name': 'send_email_tool', 'args': {'receipient': 'john@test.com', 'subject': 'How are you?', 'body': 'How are you?'}, 'id': 'call_GtiIYeeemYI

# Reject


In [37]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

def read_email_tool(email_id:str):
    """Mock function to read an email by its id"""
    return f"Email content for {email_id}"

def send_email_tool(receipient:str,subject:str,body:str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {receipient} with subject `{subject}`"


agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approve","edit","reject"]
                },
                "read_email_tool":False,
            }
        )
        
    ]
)

config={"configurable":{"thread_id":"test-reject"}}

#Step 1 - request
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'")]},
    config=config
)


In [38]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='800274be-c1c7-49cd-a847-49dad02a0abe'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CviyhOTCLy1KSUJhja2H2O4EAaiSW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d7a-c394-7e80-ab0a-d8005dd99480-0', tool_calls=[{'name': 'send_email_tool', 'args': {'receipient': 'john@test.com', 'subject': 'How are you?', 'body': 'How are you?'}, 'id': 'call_37HAnCCjVJ0

In [39]:
from langgraph.types import Command
# Step 2 Aprove
if "__interrupt__" in result:
    print("Paused! Approving..")
    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    {"type":"reject"}
                ]
        }),
        config=config
    )

    print(f"Result : {result['messages'][-1].content}")


Paused! Approving..
Result : It seems that there was an issue sending the email. Would you like me to try again or assist you with something else?


In [40]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='800274be-c1c7-49cd-a847-49dad02a0abe'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CviyhOTCLy1KSUJhja2H2O4EAaiSW', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d7a-c394-7e80-ab0a-d8005dd99480-0', tool_calls=[{'name': 'send_email_tool', 'args': {'receipient': 'john@test.com', 'subject': 'How are you?', 'body': 'How are you?'}, 'id': 'call_37HAnCCjVJ0

## Editing

In [41]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from  langchain_core.messages import HumanMessage,SystemMessage

def read_email_tool(email_id:str):
    """Mock function to read an email by its id"""
    return f"Email content for {email_id}"

def send_email_tool(receipient:str,subject:str,body:str) -> str:
    """Mock function to send an email."""
    return f"Email sent to {receipient} with subject `{subject}`"


agent = create_agent(
    model="gpt-4o-mini",
    tools=[read_email_tool,send_email_tool],
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool":{
                    "allowed_decisions":["approve","edit","reject"]
                },
                "read_email_tool":False,
            }
        )
        
    ]
)

config={"configurable":{"thread_id":"test-editing"}}

#Step 1 - request
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'")]},
    config=config
)


In [42]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='87041df4-2a66-46cf-9e4e-1105e6b30dbe'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cvj08P73AskOEDC364JrilsnhmsBv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d7c-1ebd-74e1-866e-bab2f2d17888-0', tool_calls=[{'name': 'send_email_tool', 'args': {'receipient': 'john@test.com', 'subject': 'How are you?', 'body': 'How are you?'}, 'id': 'call_226vNjLiZAo

In [ ]:
from langgraph.types import Command
# Step 2 Aprove
if "__interrupt__" in result:
    print("Editing! Editing..")
    result = agent.invoke(
        Command(
            resume={
                "decisions":[
                    {
                        "type":"edit",
                        "edited_action":{
                            "name":"send_email_tool",
                            "args":{
                                "receipient":"correct@gmail.com",
                                "subject":"Corrected Subject",
                                "body":"This was edited by human before editing"
                            }
                        }
                    }
                ]
        }),
        config=config
    )

    print(f"Result : {result['messages'][-1].content}")


Paused! Approving..
Result : 


In [44]:
result

{'messages': [HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'", additional_kwargs={}, response_metadata={}, id='87041df4-2a66-46cf-9e4e-1105e6b30dbe'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 95, 'total_tokens': 127, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-Cvj08P73AskOEDC364JrilsnhmsBv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b9d7c-1ebd-74e1-866e-bab2f2d17888-0', tool_calls=[{'type': 'tool_call', 'name': 'send_email_tool', 'args': {'receipient': 'correct@gmail.com', 'subject': 'Corrected Subject', 'body': 'This was 

## Logging Middleware

In [45]:
from langchain.agents.middleware import (
    AgentMiddleware,
    AgentState,
    ModelRequest,
    ModelResponse,
)
from langgraph.runtime import Runtime
from typing import Any, Callable

class LoggingMiddleware(AgentMiddleware):
    def before_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"About to call model with {len(state['messages'])} messages")
        return None

    def after_model(self, state: AgentState, runtime: Runtime) -> dict[str, Any] | None:
        print(f"Model returned: {state['messages'][-1].content}")
        return None

In [49]:
agent = create_agent(
    model="gpt-4o-mini",
    middleware=[LoggingMiddleware()],
    tools=[],
)


In [48]:
config={"configurable":{"thread_id":"test-logging"}}

#Step 1 - request
result = agent.invoke(
    {"messages":[HumanMessage(content="Send email to john@test.com with subject and body 'How are you?'")]},
    config=config
)


About to call model with 1 messages
Model returned: I can't send emails directly, but I can help you draft one! Here's a simple email template you can use:

---

**To:** john@test.com  
**Subject:** How are you?

**Body:**  
Hi John,

How are you?

Best regards,  
[Your Name]

---

Feel free to customize it as needed!
